<a href="https://colab.research.google.com/github/avlis-MMO/Deep_Learning/blob/main/Project_5_Image_Classification/CNN2_Intel_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install seaborn
!pip install matplotlib
!pip install opencv-python

# **Import data**

In [ ]:
import tensorflow  as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import cv2
import random
from sklearn.utils import shuffle

In [ ]:

folderPath_train = "/content/drive/MyDrive/archive/CNN2_Intel_Classification/seg_train/seg_train"
PathList = os.listdir(folderPath_train)

data_train = []
label_train = []

IMAGE_SIZE = (100, 100)

class_names = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

for i, path in enumerate(PathList):
  for img in os.listdir(os.path.join(folderPath_train, path)):
    label_train.append(i)
    image = cv2.imread(os.path.join(os.path.join(folderPath_train, path),img))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, IMAGE_SIZE)
    data_train.append(image)

data_train = np.array(data_train)
label_train = np.array(label_train)

folderPath_test = "/content/drive/MyDrive/archive/CNN2_Intel_Classification/seg_test/seg_test"
PathList = os.listdir(folderPath_test)

data_test = []
label_test = []

for i, path in enumerate(PathList):
  for img in os.listdir(os.path.join(folderPath_test, path)):
    label_test.append(i)
    image = cv2.imread(os.path.join(os.path.join(folderPath_test, path),img))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, IMAGE_SIZE)
    data_test.append(image)

data_test = np.array(data_test)
label_test = np.array(label_test)

print(len(data_train))

# **Visualize data**

In [ ]:
n_train = label_train.shape[0]
n_test = label_test.shape[0]

print (f"Number of training examples: {n_train}")
print (f"Number of testing examples: {n_test}")
print (f"Each image is of size: {IMAGE_SIZE}")


_, train_unique = np.unique(label_train, return_counts=True)
_, test_unique = np.unique(label_test, return_counts=True)
print(label_train[5])


data_train, label_train = shuffle(data_train, label_train, random_state=10)


pd.DataFrame({'train': train_unique,
                    'test': test_unique},
             index=class_names
            ).plot.bar()
plt.show()

fig = plt.figure(figsize=(10,10))
fig.suptitle("Some examples of images of the dataset", fontsize=16)

rand_n = random.sample(range(1000, 13000), 25)
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(data_train[rand_n[i]], cmap=plt.cm.binary)
    plt.xlabel(class_names[label_train[rand_n[i]]])
plt.show()

# **Build Model**

In [ ]:
data_train = data_train /255
data_test = data_test /255

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (5,5), 2, activation ='relu', input_shape = (100,100,3)),
    tf.keras.layers.MaxPool2D(3,3),
    tf.keras.layers.Conv2D(32, (3,3), 1, activation ='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64, (2,2), 1, activation ='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(6, activation ='softmax')

])



# **Compile model**

In [ ]:
def lr_schedule(epoch, lr):
    if epoch < 25:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

initial_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
callbacks = [lr_scheduler]

# **Train model**

In [ ]:
history = model.fit(data_train, label_train, batch_size=64, epochs=50, validation_split = 0.25, callbacks=callbacks)

fig = plt.figure(figsize=(10,5))

# Plot accuracy
plt.subplot(221)
plt.plot(history.history['accuracy'],'bo--', label = "accuracy")
plt.plot(history.history['val_accuracy'], 'ro--', label = "val_accuracy")
plt.title("train_accuracy vs val_accuracy")
plt.ylabel("accuracy")
plt.xlabel("epochs")
plt.legend()

# Plot loss function
plt.subplot(222)
plt.plot(history.history['loss'],'bo--', label = "loss")
plt.plot(history.history['val_loss'], 'ro--', label = "val_loss")
plt.title("train_loss vs val_loss")
plt.ylabel("loss")
plt.xlabel("epochs")

plt.legend()
plt.show()

# **Test model**

In [ ]:
test_loss = model.evaluate(data_test, label_test)

# **Agg data**

In [ ]:
# Data agumentation to avoid overfitting

data_agg = tf.keras.Sequential([
                                tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal", input_shape = (100,100,3)),
                                tf.keras.layers.experimental.preprocessing.RandomRotation(0.05),
                                tf.keras.layers.experimental.preprocessing.RandomZoom(0.05),

])



In [ ]:
model = tf.keras.Sequential([
    data_agg,
    tf.keras.layers.Conv2D(16, (5,5), 2, activation ='relu', input_shape = (100,100,3)),
    tf.keras.layers.MaxPool2D(3,3),
    tf.keras.layers.Conv2D(32, (3,3), 1, activation ='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64, (2,2), 1, activation ='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(6, activation ='softmax')

])




In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
callbacks = [lr_scheduler]

In [ ]:
history = model.fit(data_train, label_train, batch_size=64, epochs=50, validation_split = 0.25, callbacks=callbacks)

fig = plt.figure(figsize=(10,5))

# Plot accuracy
plt.subplot(221)
plt.plot(history.history['accuracy'],'bo--', label = "accuracy")
plt.plot(history.history['val_accuracy'], 'ro--', label = "val_accuracy")
plt.title("train_accuracy vs val_accuracy")
plt.ylabel("accuracy")
plt.xlabel("epochs")
plt.legend()

# Plot loss function
plt.subplot(222)
plt.plot(history.history['loss'],'bo--', label = "loss")
plt.plot(history.history['val_loss'], 'ro--', label = "val_loss")
plt.title("train_loss vs val_loss")
plt.ylabel("loss")
plt.xlabel("epochs")

plt.legend()
plt.show()

In [ ]:
test_loss = model.evaluate(data_test, label_test)